In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, savefig
import time
import csv
import sys
import plotly.express as px

In [2]:
df = pd.read_csv("/Users/hamin/bithumb_bot_data/include/up-btc/upbit-2019-02-btc-krw.csv")

In [3]:
df.dtypes

timestamp     object
quantity     float64
price          int64
fee          float64
amount         int64
side           int64
dtype: object

In [4]:
## converting timestamp column type 'object' to 'datetime'
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df.dtypes

timestamp    datetime64[ns]
quantity            float64
price                 int64
fee                 float64
amount                int64
side                  int64
dtype: object

In [5]:
## converting timestamp to detailed date column
df['timestamp_year_month_day']       = df['timestamp'].dt.date         # YYYY-MM-DD(문자)
df['timestamp_year']       = df['timestamp'].dt.year         # 연(4자리숫자)
df['timestamp_month']      = df['timestamp'].dt.month        # 월(숫자)
df['timestamp_month_name'] = df['timestamp'].dt.month_name() # 월(문자)

df['timestamp_day']        = df['timestamp'].dt.day          # 일(숫자)
df['timestamp_time']       = df['timestamp'].dt.time         # HH:MM:SS(문자)
df['timestamp_hour']       = df['timestamp'].dt.hour         # 시(숫자)
df['timestamp_minute']     = df['timestamp'].dt.minute       # 분(숫자)
df['timestamp_second']     = df['timestamp'].dt.second       # 초(숫자)

In [6]:
df

,timestamp,quantity,price,fee,amount,side,timestamp_year_month_day,timestamp_year,timestamp_month,timestamp_month_name,timestamp_day,timestamp_time,timestamp_hour,timestamp_minute,timestamp_second
0,2019-02-21 13:55:00,0.032700,4345000,71.04,-142153,0,2019-02-21,2019,2,February,21,13:55:00,13,55,0
1,2019-02-21 13:57:00,0.032700,4350000,71.12,142173,1,2019-02-21,2019,2,February,21,13:57:00,13,57,0
2,2019-02-21 14:14:00,3.000000,4345000,6517.50,-13041518,0,2019-02-21,2019,2,February,21,14:14:00,14,14,0
3,2019-02-21 14:15:00,0.248322,4348000,539.85,1079163,1,2019-02-21,2019,2,February,21,14:15:00,14,15,0
4,2019-02-21 14:15:00,0.757456,4348000,1646.70,3291771,1,2019-02-21,2019,2,February,21,14:15:00,14,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4574,2019-02-28 20:50:00,0.112846,4254000,240.02,479804,1,2019-02-28,2019,2,February,28,20:50:00,20,50,0
4575,2019-02-28 20:50:00,0.939000,4254000,1997.25,3992508,1,2019-02-28,2019,2,February,28,20:50:00,20,50,0
4576,2019-02-28 20:50:00,0.500000,4253000,1063.25,2125436,1,2019-02-28,2019,2,February,28,20:50:00,20,50,0
4577,2019-02-28 20:50:00,0.470021,4253000,999.50,1998001,1,2019-02-28,2019,2,February,28,20:50:00,20,50,0


In [7]:
## creating 'day - amount' connection
group_day_amount = df['amount'].groupby(df['timestamp_year_month_day'])

In [8]:
group_day_amount.size()

timestamp_year_month_day
2019-02-21     664
2019-02-22     561
2019-02-23      66
2019-02-24    1006
2019-02-25     637
2019-02-26     484
2019-02-27     500
2019-02-28     661
Name: amount, dtype: int64

In [9]:
group_day_amount

In [10]:
## 일별 총 손익량
group_day_amount.sum()

timestamp_year_month_day
2019-02-21   -12735738
2019-02-22     -414093
2019-02-23    13044892
2019-02-24   -12536287
2019-02-25   -11850525
2019-02-26    13326461
2019-02-27    -1404417
2019-02-28    12434234
Name: amount, dtype: int64

In [11]:
## creating 'day - earning amount' connection
df2 = df.copy()
df2 =df2[df2['amount'] > 0]
group_day_amount_positive = df2['amount'].groupby(df2['timestamp_year_month_day'])


In [12]:
# 일별 수익 transaction 수
group_day_amount_positive.size()

timestamp_year_month_day
2019-02-21    332
2019-02-22    164
2019-02-23     38
2019-02-24    319
2019-02-25    445
2019-02-26    358
2019-02-27    425
2019-02-28    523
Name: amount, dtype: int64

In [13]:
# 일별 수익량
group_day_amount_positive.sum()

timestamp_year_month_day
2019-02-21    235153054
2019-02-22    141382200
2019-02-23     27070007
2019-02-24    431903008
2019-02-25    460287791
2019-02-26    514769459
2019-02-27    253155979
2019-02-28    352148551
Name: amount, dtype: int64

In [14]:
## creating 'day - loss amount' connection
df3 = df.copy()
df3 = df3[df3['amount'] < 0]
group_day_amount_negative = df3['amount'].groupby(df3['timestamp_year_month_day'])

In [15]:
# 일별 손해 transaction 수
group_day_amount_negative.size()

timestamp_year_month_day
2019-02-21    332
2019-02-22    397
2019-02-23     28
2019-02-24    687
2019-02-25    192
2019-02-26    124
2019-02-27     75
2019-02-28    137
Name: amount, dtype: int64

In [16]:
# 일별 손해량
group_day_amount_negative.sum()

timestamp_year_month_day
2019-02-21   -247888792
2019-02-22   -141796293
2019-02-23    -14025115
2019-02-24   -444439295
2019-02-25   -472138316
2019-02-26   -501442998
2019-02-27   -254560396
2019-02-28   -339714317
Name: amount, dtype: int64

In [17]:
# 일별 절대값 총합
absolute_negative_sum=group_day_amount_negative.sum()
absolute_negative_sum = absolute_negative_sum * (-1)
absolute_total_amout=absolute_negative_sum+group_day_amount_positive.sum()
absolute_total_amout

timestamp_year_month_day
2019-02-21     483041846
2019-02-22     283178493
2019-02-23      41095122
2019-02-24     876342303
2019-02-25     932426107
2019-02-26    1016212457
2019-02-27     507716375
2019-02-28     691862868
Name: amount, dtype: int64

In [18]:
## 일별 계산 table 생성
dfs = [group_day_amount.size(), group_day_amount.sum(), group_day_amount_positive.size(), group_day_amount_positive.sum(), group_day_amount_negative.size(), group_day_amount_negative.sum(), absolute_total_amout, absolute_negative_sum]
dfs

[timestamp_year_month_day
 2019-02-21     664
 2019-02-22     561
 2019-02-23      66
 2019-02-24    1006
 2019-02-25     637
 2019-02-26     484
 2019-02-27     500
 2019-02-28     661
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2019-02-21   -12735738
 2019-02-22     -414093
 2019-02-23    13044892
 2019-02-24   -12536287
 2019-02-25   -11850525
 2019-02-26    13326461
 2019-02-27    -1404417
 2019-02-28    12434234
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2019-02-21    332
 2019-02-22    164
 2019-02-23     38
 2019-02-24    319
 2019-02-25    445
 2019-02-26    358
 2019-02-27    425
 2019-02-28    523
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2019-02-21    235153054
 2019-02-22    141382200
 2019-02-23     27070007
 2019-02-24    431903008
 2019-02-25    460287791
 2019-02-26    514769459
 2019-02-27    253155979
 2019-02-28    352148551
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2019-02-21    332
 2019-02-22    397
 2019-02-23 

In [19]:
day_df_final = reduce(lambda left,right: pd.merge(left,right,on='timestamp_year_month_day'), dfs)
day_df_final.columns = [ "daily transaction", "daily income", "daily Yield transaction", "daily Yield", "daily loss transaction", "daily loss", "daily absolute amount", "daily absolute negative amount"]
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2019-02-21,664,-12735738,332,235153054,332,-247888792,483041846,247888792
2019-02-22,561,-414093,164,141382200,397,-141796293,283178493,141796293
2019-02-23,66,13044892,38,27070007,28,-14025115,41095122,14025115
2019-02-24,1006,-12536287,319,431903008,687,-444439295,876342303,444439295
2019-02-25,637,-11850525,445,460287791,192,-472138316,932426107,472138316
2019-02-26,484,13326461,358,514769459,124,-501442998,1016212457,501442998
2019-02-27,500,-1404417,425,253155979,75,-254560396,507716375,254560396
2019-02-28,661,12434234,523,352148551,137,-339714317,691862868,339714317


In [20]:
print(day_df_final.asfreq('D'))

                          daily transaction  daily income  \
timestamp_year_month_day                                    
2019-02-21                              664     -12735738   
2019-02-22                              561       -414093   
2019-02-23                               66      13044892   
2019-02-24                             1006     -12536287   
2019-02-25                              637     -11850525   
2019-02-26                              484      13326461   
2019-02-27                              500      -1404417   
2019-02-28                              661      12434234   

                          daily Yield transaction  daily Yield  \
timestamp_year_month_day                                         
2019-02-21                                    332    235153054   
2019-02-22                                    164    141382200   
2019-02-23                                     38     27070007   
2019-02-24                                    319    431903

In [21]:
day_df_final.to_csv("/Users/hamin/bithumb_bot_data/upbit-2019-02-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')


In [22]:
# 뒤집기.
day_df_final[::-1].to_csv("/Users/hamin/bithumb_bot_data/upbit-2019-02-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')

In [23]:
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-01-19,40,1052348,21,1334332,19,-281984,1616316.0,281984
2018-01-20,153,426,76,1239191,77,-1238765,2477956.0,1238765
2018-01-21,154,-1741,76,1201705,78,-1203446,2405151.0,1203446
2018-01-22,69,-14922,34,480676,35,-495598,976274.0,495598
2018-01-23,96,899,48,633072,48,-632173,1265245.0,632173
2018-01-24,86,-1680,42,565300,44,-566980,1132280.0,566980
2018-01-25,217,11247,109,1442316,108,-1431069,2873385.0,1431069
2018-01-31,4,-86,2,22176,2,-22262,44438.0,22262
